In [32]:
# imported dependencies
import pennylane as qml
from itertools import product
import numpy as np

In [57]:
from pennylane.pauli import pauli_group
n_qubits = 2
for p in pauli_group(n_qubits):

    matrix = qml.matrix(p)
    identityMatrix = -np.identity(matrix.shape[0])
    # isNegative = np.allclose(matrix,identityMatrix)
    # if isNegative:
    #     print("wow")
    #print(qml.matrix(p))
    print(p)


Identity(wires=[0])
PauliZ(wires=[1])
PauliZ(wires=[0])
PauliZ(wires=[0]) @ PauliZ(wires=[1])
PauliX(wires=[1])
PauliY(wires=[1])
PauliZ(wires=[0]) @ PauliX(wires=[1])
PauliZ(wires=[0]) @ PauliY(wires=[1])
PauliX(wires=[0])
PauliX(wires=[0]) @ PauliZ(wires=[1])
PauliY(wires=[0])
PauliY(wires=[0]) @ PauliZ(wires=[1])
PauliX(wires=[0]) @ PauliX(wires=[1])
PauliX(wires=[0]) @ PauliY(wires=[1])
PauliY(wires=[0]) @ PauliX(wires=[1])
PauliY(wires=[0]) @ PauliY(wires=[1])


In [483]:
## 2-qubit Pauli Group
# Encode 1 logical qubit into 2 physical qubits
S1 = qml.PauliX(wires=[1]) @ qml.PauliX(wires=[2])
#S2 = qml.Identity(wires=[0]) @ qml.Identity(wires=[0])
#qml.is_commuting(S1,S2)
# Stabilizer group aka bodyguard
# S = [qml.matrix(S1),qml.matrix(S2)]
# P = 1/2 * (S[0] + S[1])
# P = qml.Hermitian(P,wires=[0,1])
G1 = S1
print(qml.matrix(G1))

[[0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]


In [9]:
dev = qml.device("default.mixed", wires=3, shots=1)



@qml.qnode(dev)
def hadamard_test(wires):
    #def create_entangled_state(wires):
    qml.PauliX(wires=[1])
    qml.PauliX(wires=[1])
    qml.PauliY(wires=[2])
    qml.PauliY(wires=[2])
    
    #qml.PauliZ(wires=wires[1])
    #qml.PauliZ(wires=wires[2])
    qml.Hadamard(wires=wires[0])
    #qml.PhaseFlip(1,wires=[0])
    #create_entangled_state(wires)
    #qml.Hadamard(wires=wires[2])
    qml.ctrl(qml.PauliZ,control=wires[0])(wires=[1])
    qml.ctrl(qml.PauliZ,control=wires[0])(wires=[2])
    return qml.expval(qml.PauliX(0) @ qml.PauliX(1))
    return qml.expval(qml.PauliX(0)), qml.expval(qml.PauliX(1)), qml.expval(qml.PauliX(2))
    #return qml.expval(qml.PauliX(0))
    #return qml.measure(qml.PauliX(0))
    #return qml.sample(qml.PauliZ(wires=0))
    #return qml.measure(qml.PauliX(0))
    #return qml.measure(qml.PauliX(0))
print(qml.draw(hadamard_test)([0,1,2]))

#create_entangled_state(wires=[0,1,2])

# sum = 0
# sum2 = 0
# #sum3 = 0
# for i in range(0,1000):
#     a,b = hadamard_test(wires=[0,1])
#     sum += a
#     sum2 += b
#     #sum3 += c
# print(sum)
# print(sum2)
# #print(sum3)
hadamard_test(wires=[0,1,2])

#p = create_entangled_state([0])
#num = np.matmul(np.matmul(P,p),P)
# den = np.sum(np.matmul(p,P))
# pDet = num/den
# print(p)
# print(pDet)


0: ──H────╭●─╭●─┤ ╭<X@X>
1: ──X──X─╰Z─│──┤ ╰<X@X>
2: ──Y──Y────╰Z─┤       


tensor(-1., requires_grad=True)

In [3]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def asdf():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    return qml.state()

result = asdf()
print(result)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [493]:
@qml.qnode(qml.device('lightning.qubit', wires=2))
def circuit():
    qml.Hadamard(0)
    qml.CRX(5, wires=[0, 1])
    qml.Rot([1,2,3], wires=[1])
    qml.CRX(7, wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [494]:
print(qml.draw(circuit))

<function circuit at 0x0000021B287BB160>


In [538]:
import pennylane as qml

n_qubits = 2  # number of physical qubits
k_logical = 1  # number of logical qubits
G1 = [qml.PauliX(0) @ qml.PauliX(1)]  # stabilizer generator

dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def test():
    # apply Hadamard gates to all qubits
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # measure stabilizer generator G1
    for i in range(k_logical):
        qml.PauliZ(wires=range(n_qubits)) @ qml.Hadamard(wires=n_qubits+i)

    # measure in the computational basis
    return [qml.sample(qml.PauliZ(i)) for i in range(n_qubits)]

# run the circuit and print the measurement results
measurement_results = syndrome_measurement_circuit()
print(measurement_results)

ValueError: PauliZ: wrong number of wires. 2 wires given, 1 expected.